In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction

In [2]:
#Función para calcular el resultado de aplicar Controlled-U_a (con a=5) un total de 'power' veces

def c_amod15(a, power):
    """Multiplicación controlada por un mod 21"""
    U = QuantumCircuit(5)
    for _iteration in range(power):
        U.swap(2,4)
        U.swap(0,2)
        U.x(0)
        U.x(2)
        U.x(4)  
    U = U.to_gate()
    U.name = f"{a}^{power} mod 21"
    c_U = U.control()
    return c_U

In [3]:
# Specify variables
N_COUNT = 14  # number of counting qubits
a = 5

In [4]:
def qft_dagger(n):
    """Inversa de la transformada cuántica de Fourier en un espacio de n qubits"""
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

In [5]:
# Crear QuantumCircuit con N_COUNT qubits
# mas 5 qubits en los que actuará U
qc = QuantumCircuit(N_COUNT + 5, N_COUNT)

# Inicializar los N_COUNT qubits en el estado |+>
for q in range(N_COUNT):
    qc.h(q)

# Negar uno de los qubtis donde actuará U para iniciar con |1>
qc.x(N_COUNT)

# Hacer las operaciones Controlled-U
for q in range(N_COUNT):
    qc.append(c_amod15(a, 2**q),
             [q] + [i+N_COUNT for i in range(5)])

# Aplicar la inversa de QFT
qc.append(qft_dagger(N_COUNT), range(N_COUNT))

# Medir
qc.measure(range(N_COUNT), range(N_COUNT))
qc.draw(fold=-1)  # -1 means 'do not fold'

┌───┐                                                                                                                                                                                                                                       ┌────────┐┌─┐                                       
 q_0: ┤ H ├───────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤0       ├┤M├───────────────────────────────────────
      ├───┤       │                                                                                                                                                                                                                               │        │└╥┘┌─┐                                    
 q_1: ┤ H ├───────┼──────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤1       ├─╫─┤M├────────────────────────────────────
      ├───┤       │              │                                                                                                                                                                                                                │        │ ║ └╥┘┌─┐                                 
 q_2: ┤ H ├───────┼──────────────┼──────────────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤2       ├─╫──╫─┤M├─────────────────────────────────
      ├───┤       │              │              │                                                                                                                                                                                                 │        │ ║  ║ └╥┘┌─┐                              
 q_3: ┤ H ├───────┼──────────────┼──────────────┼──────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤3       ├─╫──╫──╫─┤M├──────────────────────────────
      ├───┤       │              │              │              │                                                                                                                                                                                  │        │ ║  ║  ║ └╥┘┌─┐                           
 q_4: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤4       ├─╫──╫──╫──╫─┤M├───────────────────────────
      ├───┤       │              │              │              │              │                                                                                                                                                                   │        │ ║  ║  ║  ║ └╥┘┌─┐                        
 q_5: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────┼───────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤5       ├─╫──╫──╫──╫──╫─┤M├────────────────────────
      ├───┤       │              │              │              │              │               │                                                                                                                                                   │        │ ║  ║  ║  ║  ║ └╥┘┌─┐                     
 q_6: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────┼───────────────┼───────────────■────────────────────────────────────────────────────────────

In [11]:
# Exportar el circuito como imagen
circuit_diagram = qc.draw(output="mpl", fold=-1)  # fold=-1 para evitar división en varias líneas
circuit_diagram.savefig("circuit_diagram.png")  # Guardar como archivo PNG
plt.show()  # Mostrar el diagrama en pantalla

In [6]:
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

# Crear una instancia del simulador
aer_sim = AerSimulator()

# Transpile y ejecuta el circuito
t_qc = transpile(qc, aer_sim)
counts = aer_sim.run(t_qc).result().get_counts()

# Exportar el histograma como una imagen
histogram = plot_histogram(counts, figsize=(30, 15))  # Ajusta el tamaño en el histograma directamente
histogram.savefig("histogram_output.png")  # Guarda la imagen en un archivo
plt.show(histogram)  # Muestra el histograma

In [15]:
rows, measured_phases = [], []
for output in counts:
    decimal = int(output, 2)  # Convertir binario a decimal
    phase = decimal / (2 ** N_COUNT)  # Hallar el autovalor correspondiente
    measured_phases.append(phase)
    # Agregar estos valores a las filas en la tabla:
    rows.append([f"{output}(bin) = {decimal:>3}(dec)",
                  f"{decimal}/{2 ** N_COUNT} = {phase:.2f}"])

# Crear un dataframe
headers1 = ["Register Output", "Phase"]
df1 = pd.DataFrame(rows, columns=headers1)
print(df1)

                     Register Output               Phase
0    00101010101010(bin) = 2730(dec)   2730/16384 = 0.17
1   10101010101001(bin) = 10921(dec)  10921/16384 = 0.67
2    01010101100010(bin) = 5474(dec)   5474/16384 = 0.33
3   11010101001111(bin) = 13647(dec)  13647/16384 = 0.83
4   11010101010101(bin) = 13653(dec)  13653/16384 = 0.83
5     00000000000000(bin) =   0(dec)      0/16384 = 0.00
6    00101010101011(bin) = 2731(dec)   2731/16384 = 0.17
7   11010110100001(bin) = 13729(dec)  13729/16384 = 0.84
8   10101010100101(bin) = 10917(dec)  10917/16384 = 0.67
9    01010101010110(bin) = 5462(dec)   5462/16384 = 0.33
10  10101010110101(bin) = 10933(dec)  10933/16384 = 0.67
11   01010101010101(bin) = 5461(dec)   5461/16384 = 0.33
12  10101010101011(bin) = 10923(dec)  10923/16384 = 0.67
13   00101010101110(bin) = 2734(dec)   2734/16384 = 0.17
14   01010101010111(bin) = 5463(dec)   5463/16384 = 0.33
15   10000000000000(bin) = 8192(dec)   8192/16384 = 0.50
16  10101010000100(bin) = 10884

In [16]:
# Ahora calculamos el orden
rows = []
for phase in measured_phases:
    frac = Fraction(phase).limit_denominator(15)
    rows.append([phase,
                 f"{frac.numerator}/{frac.denominator}",
                 frac.denominator])

# Creamos un dataframe
headers2 = ["Phase", "Fraction", "Guess for r"]
df2 = pd.DataFrame(rows, columns=headers2)
print(df2)

       Phase Fraction  Guess for r
0   0.166626      1/6            6
1   0.666565      2/3            3
2   0.334106      1/3            3
3   0.832947      5/6            6
4   0.833313      5/6            6
5   0.000000      0/1            1
6   0.166687      1/6            6
7   0.837952      5/6            6
8   0.666321      2/3            3
9   0.333374      1/3            3
10  0.667297      2/3            3
11  0.333313      1/3            3
12  0.666687      2/3            3
13  0.166870      1/6            6
14  0.333435      1/3            3
15  0.500000      1/2            2
16  0.664307      2/3            3
17  0.833374      5/6            6
18  0.833008      5/6            6
19  0.832764      5/6            6
20  0.666443      2/3            3
21  0.166748      1/6            6
22  0.166992      1/6            6
23  0.333252      1/3            3
24  0.666748      2/3            3
25  0.166809      1/6            6
26  0.666809      2/3            3
27  0.833740      5/

In [17]:
#Exportamos los resultados a un excel
combined_df = pd.concat([df1, df2], axis=1)
combined_df.to_excel('results.xlsx', index=False)

print("Combined DataFrame exported to Excel successfully.")

Combined DataFrame exported to Excel successfully.
